In [ ]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist, euclidean

In [ ]:
import mesonic

In [ ]:
import sc3nb as scn

In [ ]:
%matplotlib widget

# Data Preparation

In [ ]:
eeg_data = np.loadtxt("../notebooks/data/epileptic-eeg.csv", delimiter=",")
eeg_df = pd.DataFrame(eeg_data)
eeg_df

In [ ]:
penguins_df = sns.load_dataset("penguins")
penguins_df = penguins_df.dropna(subset=["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g", "sex"])
penguins_df = penguins_df.reset_index(drop=True)
penguins_df

In [ ]:
seaice_df = sns.load_dataset("seaice")
seaice_df

In [ ]:
seaice_df.set_index("Date").plot()

# Sonecules

In [ ]:
import uuid

In [ ]:
context = mesonic.create_context()

In [ ]:
sonecules_default_context = context

In [ ]:
uuid.uuid4()

In [ ]:
pb = context.create_playback()

In [ ]:
context.processor is pb.processor

In [ ]:
class SoneculeFilter:
    
    def __init__(self):
        self.deactivated_sonecules = set()
    
    def __call__(self, event):
        sonecule_id = event.info.get("sonecule_id", None) 
        if sonecule_id is not None and sonecule_id in self.deactivated_sonecules:
            return None
        else:
            return event
    

In [ ]:
context.processor.event_filter = SoneculeFilter() 
# TODO this should be a list/set of filters so we don't block custom filters

In [ ]:
class Sonecule:
    
    def __init__(self, context=None): 
        if context is None:
            context = sonecules_default_context
        self.context = context
        self._sonecule_id = uuid.uuid4()
        
    # require or better ensure each synth created in a sonecule gets the
    # metadata set to include the sonecule_id so it can be filtere
    
    @property
    def sonecule_id(self):
        return self._sonecule_id
    
    @property
    def active(self, value):
        return self.sonecule_id in context.processor.event_filter.deactivated_sonecules
    
    @active.setter
    def active(self, value):
        assert isinstance(value, bool)
        if value:  # this sonecule should not be part of the timeline
            context.processor.event_filter.deactivated_sonecules.add(self.sonecule_id)
        else:
            context.processor.event_filter.deactivated_sonecules.discard(self.sonecule_id)
    
    def reset(self): # clear this 
        """iterate over timeline and remove events belonging to this Sonecule"""
        # TODO this should be done in mesonic
        ...
    

# Sonification Classes

## Score-based / Timeline Sonifications

* schedule method for score / timeline generation

In [ ]:
from sc3nb import linlin, midicps, cpsmidi

In [ ]:
s2 = context.synths.create("s2", track=1) 

Question: create Synths in sonecule or outside?
- inside: 
    - we have full control over synth creation
    - some sonecules will rely on specific synths
    - we can better provide sensible defaults
    - requires atleast a small inheritance overwritting synth name string for synth changes, perhaps also needs to redo setup
- outside:
    - user can already set defaults and bounds
    - user can adapt a sonecule quickly without much code
    


In [ ]:
s2.metadata.update({"sonecule_id": 1321})

In [ ]:
s2.metadata

In [ ]:
s2.params # param bounds would provide a target mapping bound 

### standard Discrete PMSon - old

In [ ]:
class StandardDiscretePMSon(Sonecule):
    
    def __init__(self, synth_name="s2", context=None):
        super().__init__(context)
        # if instanceof(synth,Synth): 
        self.synth = self.context.synths.create(synth_name, track=1, mutable=False)
        # creation here should add metadata which then is added to each produced event
        
    def schedule(self, df, mapping, at=0, stop_after=0.1, **odfkwargs):  # odfkwargs is a bad name    
        self.reset()
        
        dfkwargs = {"dmin": df.min(), "dmax": df.max()}
        dfkwargs.update(odfkwargs)  # allow overwriting of df
        
        for idx in df.index:
            col, fun, mkwargs = mapping["onset"]
            value = getattr(df, col)[idx]
            data_min, data_max = dfkwargs["dmin"][col], dfkwargs["dmax"][col]  # name?
            onset = fun(value, **mkwargs, dmin=data_min, dmax=data_max)
            with context.at(at+onset, info={"sonecule_id": self.sonecule_id}) as tp:
                for param in [param for param in mapping.keys() if param != "onset"]:
                    col, fun, mkwargs = mapping[param]
                    value = getattr(df, col)[idx]
                    dmin, dmax = dfkwargs["dmin"][col], dfkwargs["dmax"][col] 
                    setattr(self.synth, param, fun(value, **mkwargs, dmin=dmin, dmax=dmax))
                

### standard Continuous PMSon - old

In [ ]:
class StandardContinuousPMSon(Sonecule):
    
    def __init__(self, synth_name, context=None):
        super().__init__(context)
        # if instanceof(synth,Synth): 
        self.synth = self.context.synths.create(synth_name, track=1, mutable=True)
        # creation here should add metadata which then is added to each produced event
        
    def schedule(self, df, mapping, at=0, time_after_last=0.1, **odfkwargs):  # odfkwargs is a bad name    
        self.reset()
        
        dfkwargs = {"dmin": df.min(), "dmax": df.max()}  # TODO names for dmin, dmax?
        dfkwargs.update(odfkwargs)  # allow overwriting of df
    
        with context.at(at, info={"sonecule_id": self.sonecule_id}):
            self.synth.start()
            
        for idx in df.index:
            col, fun, mkwargs = mapping["onset"]
            value = getattr(df, col)[idx]
            data_min, data_max = dfkwargs["dmin"][col], dfkwargs["dmax"][col]
            onset = fun(value, **mkwargs, dmin=data_min, dmax=data_max)
            with context.at(at+onset, info={"sonecule_id": self.sonecule_id}) as tp:
                for param in [param for param in mapping.keys() if param != "onset"]:
                    col, fun, mkwargs = mapping[param]
                    value = getattr(df, col)[idx]
                    dmin, dmax = dfkwargs["dmin"][col], dfkwargs["dmax"][col]
                    value = fun(value, **mkwargs, dmin=dmin, dmax=dmax)
                    setattr(self.synth, param, value)
                
        with context.at(at + onset + time_after_last, info={"sonecule_id": self.sonecule_id}):
            self.synth.stop()



In [ ]:
scPMSon = StandardContinuousPMSon("s2")

In [ ]:
assert scPMSon.context is sonecules_default_context

In [ ]:
context.timeline.reset()

In [ ]:
context.timeline

In [ ]:
mapping = {
    "onset": 
        ("flipper_length_mm",
         lambda value, dmin, dmax, y1, y2: linlin(value, dmin, dmax, y1, y2),
         dict(y1=0, y2=5)),
    "freq": 
        ("body_mass_g",
         lambda value, dmin, dmax, y1, y2: midicps(linlin(value, dmin, dmax, y1, y2)),
         dict(y1=48, y2=25)),
    }

scPMSon.schedule(penguins_df, mapping, at=2, time_after_last=2)

In [ ]:
context.timeline

In [ ]:
context.timeline.plot()

In [ ]:
pb.start(2)

### WIP standardContinuousPMSon - new format - init with Synth Bounds and schedule with mapping_spec, data bounds 

In [ ]:
from typing import Optional, Dict, Any

In [ ]:
from  mesonic.synth import Synth 

In [ ]:
class StandardContinuousPMSon(Sonecule):
    SPECIAL_PARAMETER_SPECS = ["conversion"]
    
    
    def __init__(self, synth: str = "s2", parameter_specs: Optional[Dict[str, Dict[str, Any]]] = None, context=None):
        super().__init__(context)
        if isinstance(synth, Synth):
            assert s2.mutable, "Synth needs to be mutable for continuous Parameter Mapping Sonification"
            self.synth = synth
        else:
            self.synth = self.context.synths.create(synth, track=1)
        if parameter_specs is None:
            parameter_specs = dict()
        
        def _get_conversion(parameter_name):
            try:
                return parameter_specs[parameter_name]["conversion"]
            except KeyError:
                return None
        
        self.conversions = {param: _get_conversion(param) for param in self.synth.params}        
        
        # treat the rest f.e. bounds as synth attributes
        for param, param_spec in parameter_specs.items():
            if param not in self.synth.params:
                raise ValueError(f"{param} is not a Parameter of {self.synth}") 
            else:
                if "bounds" in param_spec:
                    getattr(self.synth, param).bounds = param_spec["bounds"]
                if "default" in param_spec:
                    getattr(self.synth, param)._default = param_spec["default"]
        
        # creation here should add metadata which then is added to each produced event
        self.synth.metadata = self.sonecule_id
        
    def schedule(self, df, mapping, at=0, stop_after=0.1, **odfkwargs):  # odfkwargs is a bad name
        # clear the current events from the timeline
        self.reset()
        
        dfkwargs = {"dmin": df.min(), "dmax": df.max()}  # TODO names for dmin, dmax?
        dfkwargs.update(odfkwargs)  # allow overwriting of df
    
        with context.at(at, info={"sonecule_id": self.sonecule_id}):
            self.synth.start()
        
        # TODO
        
        for idx in df.index:
            col, fun, mkwargs = mapping["onset"]
            value = getattr(df, col)[idx]
            data_min, data_max = dfkwargs["dmin"][col], dfkwargs["dmax"][col]
            onset = fun(value, **mkwargs, dmin=data_min, dmax=data_max)
            with context.at(at+onset, info={"sonecule_id": self.sonecule_id}) as tp:
                for param in [param for param in mapping.keys() if param != "onset"]:
                    col, fun, mkwargs = mapping[param]
                    value = getattr(df, col)[idx]
                    dmin, dmax = dfkwargs["dmin"][col], dfkwargs["dmax"][col] 
                    setattr(self.synth, param, fun(value, **mkwargs, dmin=dmin, dmax=dmax))
                
        with context.at(at + onset + stop_after, info={"sonecule_id": self.sonecule_id}):
            self.synth.stop()


In [ ]:
scpmson = StandardContinuousPMSon("s2", {"freq": {"bounds": (50,70), "conversion": midicps}, "amp": {"default": 0.1}})  # bounds are in pre conversion unit - but this is done differently here in the code

In [ ]:
scpmson.synth.params

In [ ]:
plt.plot([scn.midicps(x) for x in range(128)]);

### Time-variant Oscillator bank mapping

for multivariate time series, pitchmapping on variable nr. of independent oscillators, value -> pitch deviation from a centers, centers are musically equidistant between a minimal and maximal MIDI number. 
 - Special features: change-driven amplitude (difference mapped to amplitude to emphasize changes)
 - Special case: Auditory graph


In [ ]:
context.reset()

## Buffer Synthesis Sonifications

Sonifications that are implemented as Synthesis that acts on a Buffer

good for realtime / interactive sonifications

In [ ]:
class BufferSynth(Sonecule):
    
    def __init__(self, data, sr, context=None):
        super().__init__(context=context)
        if type(self).synth_name not in self.context.synths.buffer_synthdefs:
            raise NotImplementedError("the selected Context does not offer an {self.synth_name} Synth")
        # self.df = ...
        self.buf = self.context.buffers.from_data(data, sr)
        self.synth = self.context.synths.from_buffer(self.buf, synth_name=type(self).synth_name)
        
    def resampling(self, **kwargs):
        # data = self.buf ---- .data
        # asig = Asig(...)
        # self.buf = self.context.buffer.from_asig(asig)
        ...
        
    def schedule(self, at=0):
        with self.context.at(time=at):
            self.synth.start()
    
    def start(from_t, to_t): # slicing
        ...
    

In [ ]:
context.enable_realtime();

In [ ]:
%matplotlib qt

### Simple Audification

In [ ]:
print(context.synths.buffer_synthdefs["playbuf"])

In [ ]:
class Audification(BufferSynth):
    synth_name = 'playbuf'
    

In [ ]:
audification = Audification(data=eeg_data[:,[0,1]], sr=256)

In [ ]:
audification.synth.params

In [ ]:
audification.synth.start(rate=24)

### TimbralSon

* The timbralson uses the channels of the data to modulate the amplitude of a harmonic of the fundamental frequency `f0` for each channel.

In [ ]:
context.synths.buffer_synthdefs["timbralson"]= r"""
        { |bufnum={{BUFNUM}}, f0=90, amp=0.1, rate=1 |
            var nch = {{NUM_CHANNELS}};
            var sines = SinOsc.ar(nch.collect{|i| f0*rate*(i+1)});
            var playbufs = PlayBuf.ar(nch, bufnum, BufRateScale.kr(bufnum)*rate, doneAction: 2 ) ;
            Out.ar(0, (sines * playbufs).sum * amp!2 )
        }"""

In [ ]:
class TimbralSon(BufferSynth):  
    synth_name = 'timbralson'

# that is possible but the TimbralSon is not really an Audification but also direcly uses
# the buffer with the data in a Ugen graph

In [ ]:
timbralson = TimbralSon(eeg_data[14*256:24*256], sr=256)

In [ ]:
timbralson.synth.params

In [ ]:
timbralson.synth.start({"f0": 90, "rate": 0.5})

The created Synth will offer the Parameters defined above and we can adapt them while the Synth plays.

In [ ]:
timbralson.synth.f0 = 70

In [ ]:
timbralson.synth.f0 = 100

In [ ]:
timbralson.synth.rate = 1

In [ ]:
timbralson.synth.stop()

In [ ]:
ctx.timeline.plot()

In [ ]:
ctx.clo

## Sonification as Handler/Callback/Setup

* MBS 
* Event-based?


### Data Sonogram

The `DataSonogram` implements a Data Sonogram

- The model gets a dataset which is plotted in two dimensions.
- When the user clicks into the plot a shock wave (signaled by noise Synth) is created from the nearest data point.



In [ ]:
scn.SynthDef("noise", r"""
{ |out=0, freq=2000, rq=0.02, amp=0.3, dur=1, pos=0 |
    Out.ar(out, Pan2.ar(
        BPF.ar(WhiteNoise.ar(10), freq, rq) 
        * Line.kr(1, 0, dur, doneAction: 2).pow(4), pos, amp));
}""").add()

In [ ]:
class DataSonogram(Sonecule):

    def __init__(self, df, x, y, label, max_duration=1.5, spring_synth="s1", trigger_synth="noise", context=None):
        super().__init__(context=context)
        
        #prepare synths
        self.trigger_synth = self.context.synths.create(trigger_synth, mutable=False)  # TODO make sure the synths have metadata
        self.spring_synth = self.context.synths.create(spring_synth, mutable=False)
        
        # save dataframe
        self.df = df
        self.numeric_df = df.select_dtypes(include=[np.number])
        
        # check if x and y are valid
        allowed_columns = self.numeric_df.columns
        assert x in allowed_columns, f"x must be in {allowed_columns}"
        assert y in allowed_columns, f"y must be in {allowed_columns}"

        # prepare data for model
        self.labels = self.df[label]
        self.unique_labels = self.labels.unique()
        label2id = {label: idx for idx, label in enumerate(self.unique_labels)}
        self.numeric_labels = [label2id[label] for label in self.labels]
        self.xy_data = self.numeric_df[[x,y]].values 
        self.data = self.numeric_df.values

        # get the convex hull of the data
        hull = ConvexHull(self.data)
        hull_data = self.data[hull.vertices,:]
        # get distances of the data points in the hull 
        hull_distances = cdist(hull_data, hull_data, metric='euclidean')
        self.max_distance =  hull_distances.max()
        
        # set model parameter
        self.max_duration = max_duration
        
        # prepare plot
        self.fig = plt.figure(figsize=(5,5))
        self.ax = plt.subplot(111)

        # plot data
        sns.scatterplot(x=x, y=y, hue=label, data=df, ax=self.ax)
        
        # set callback
        def onclick(event):
            if event.inaxes is None: # outside plot area
                return
            if event.button != 1: # ignore other than left click 
                return
            click_xy = np.array([event.xdata, event.ydata])
            self.create_shockwave(click_xy)
        
        self.fig.canvas.mpl_connect('button_press_event', onclick)

    def create_shockwave(self, click_xy):
        self.context.reset()
        
        with self.context.now() as start_time:
            self.trigger_synth.start()
        # find the point that is the nearest to the click location
        center_idx = np.argmin(np.linalg.norm(self.xy_data - click_xy, axis=1))
        center = self.data[center_idx]
        # get the distances from the other points to this point
        distances_to_center = np.linalg.norm(self.data - center, axis=1)
        # get idx sorted by distances
        order_of_points = np.argsort(distances_to_center)
        # for each point create a sound using the spring synth
        for idx in order_of_points:
            distance = distances_to_center[idx]
            nlabel = self.numeric_labels[idx]
            n = len(self.unique_labels)-1
            onset = (distance / self.max_distance) * self.max_duration
            with self.context.at(start_time + onset):
                self.spring_synth.start(
                    freq = 2 * (400 + 100 * nlabel),
                    amp = scn.dbamp(scn.linlin(distance, 0, self.max_distance, -10, -30)),
                    pan = [-1,1][int(self.xy_data[idx, 0]-click_xy[0] > 0)],
                    dur = 0.04,
                    info = {"label": self.labels[idx]},
                )

In [ ]:
dsg1 = DataSonogram(penguins_df, x="flipper_length_mm", y="body_mass_g", label="species")